In [279]:
import numpy as np
import pandas as pd

from zipline.data.bundles import register
from zipline.data.bundles.csvdir import csvdir_equities

from zipline.api import symbol, order, record, fetch_csv, schedule_function
import zipline

from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.efficient_frontier import EfficientFrontier

In [155]:
%load_ext zipline

The zipline extension is already loaded. To reload it, use:
  %reload_ext zipline


In [311]:
%%zipline --start 2018-3-17 --end 2019-3-14 --bundle custom-csvdir-bundle -o play_portfolio.pickle

import numpy as np
import pandas as pd
from datetime import timedelta, date

from zipline.api import order, record, fetch_csv, symbol, order_percent, order_value, order_target_percent
from zipline.api import schedule_function, date_rules, get_datetime
from zipline.finance import commission, slippage
from zipline.data.bundles import register
from zipline.data.bundles.csvdir import csvdir_equities
from zipline.protocol import Portfolio

from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.efficient_frontier import EfficientFrontier

import zipline

def initialize(context):
    
    context.etfs = ['BIB', 'BRZU', 'CURE', 'DRN', 'EDC', 'FAS', 'FINU', 'GOEX', 'INDL','JNUG', 'JPNL', 'KOLD',
                    'LBJ','MIDU', 'NUGT', 'QLD','RETL', 'ROM','SCO', 'SOXL', 'SPXL', 'SVXY', 'TECL', 'TMF', 'TNA',
                    'TQQQ','UBR','UDOW', 'UMDD', 'UPRO', 'URTY', 'USD', 'UWM', 'XPP', 'YINN']
    
    starting_allocations = [1.0/len(context.etfs) for i in range(len(context.etfs))]
    context.allocation = 1.0/len(context.etfs)
    
    context.set_commission(commission.PerShare(cost=0.0075, min_trade_cost=0))
    #context.set_slippage(slippage.VolumeShareSlippage())
    
    optimize_df = pd.read_csv('arima_predictions.csv')
    optimize_df['date'] = pd.to_datetime(optimize_df['date'])
    optimize_df = optimize_df.set_index('date')
    #optimize_df.index = [pd.Timestamp(date(date)) for date in optimize_df['date']]
    context.predictions = optimize_df
    
    context.buy = True
    context.counter = 0
    
    fetch_csv('https://raw.githubusercontent.com/stephenilhardt/project-kojak/master/arima_preds.csv',
              date_column='date', symbol_column='symbol')
    
    schedule_function(func = reset_weights, date_rule = date_rules.every_day())
    #schedule_function(func = rebalance, date_rule = date_rules.week_start())
    
def handle_data(context, data):
    
    #if context.buy == True:
    if context.counter % 1 == 0:
        if type(context.allocation) == dict:
            for etf in context.etfs:
                order_target_percent(symbol(etf), context.allocation[etf])
        else:
            pass
        
    context.counter += 1
            
def reset_weights(context, data):
    
    current_datetime = get_datetime()
    context.week_start = current_datetime + timedelta(days=1)
    
    start_date_str = str(current_datetime).split()[0]
    start_date = pd.to_datetime(start_date_str)
    end_date = start_date + timedelta(days=30)
    
    date_range = pd.date_range(start=start_date,end=end_date,freq='D')
    
    predictions = context.predictions    
    df = pd.DataFrame(predictions[predictions.index.isin(date_range)])   

    for etf in context.etfs:
        pass
    
    mu = mean_historical_return(df)
    S = CovarianceShrinkage(df).ledoit_wolf()
    ef = EfficientFrontier(mu, S)
    weights = ef.max_sharpe()
    
    context.allocation = weights
    
    return weights

def get_historical_prices(data, context):
    df = pd.DataFrame()
    
    for etf in context.etfs:
        prices = data.history(symbol(etf), 'price', 10, '1d')
        df[etf] = prices
    
    return df

,algo_volatility,algorithm_period_return,alpha,benchmark_period_return,benchmark_volatility,beta,capital_used,ending_cash,ending_exposure,ending_value,...,short_exposure,short_value,shorts_count,sortino,starting_cash,starting_exposure,starting_value,trading_days,transactions,treasury_period_return
2018-03-19 20:00:00+00:00,NaN,0.000000,NaN,-0.013530,NaN,NaN,0.000000e+00,1.000000e+07,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0,NaN,1.000000e+07,0.000000e+00,0.000000e+00,1,[],0.0
2018-03-20 20:00:00+00:00,0.000000,0.000000,0.000000,-0.011853,0.170964,0.000000,0.000000e+00,1.000000e+07,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0,NaN,1.000000e+07,0.000000e+00,0.000000e+00,2,[],0.0
2018-03-21 20:00:00+00:00,0.005135,-0.000560,-0.060670,-0.013749,0.126315,-0.011785,-8.893866e+06,1.106134e+06,8.888264e+06,8.888264e+06,...,0.000000e+00,0.000000e+00,0,-9.165151,1.000000e+07,0.000000e+00,0.000000e+00,3,"[{'dt': 2018-03-21 20:00:00+00:00, 'amount': 3...",0.0
2018-03-22 20:00:00+00:00,0.485187,-0.061838,1.295337,-0.038403,0.192071,2.127534,6.884699e+05,1.794604e+06,7.587014e+06,7.587014e+06,...,0.000000e+00,0.000000e+00,0,-8.009444,1.106134e+06,8.888264e+06,8.888264e+06,4,"[{'dt': 2018-03-22 20:00:00+00:00, 'amount': 9...",0.0
2018-03-23 20:00:00+00:00,0.476919,-0.106163,1.302704,-0.058899,0.185697,2.247164,-4.148106e+04,1.753123e+06,7.185247e+06,7.185247e+06,...,0.000000e+00,0.000000e+00,0,-10.007811,1.794604e+06,7.587014e+06,7.587014e+06,5,"[{'dt': 2018-03-23 20:00:00+00:00, 'amount': -...",0.0
2018-03-26 20:00:00+00:00,0.683719,-0.051974,0.902029,-0.033151,0.304452,2.139679,-4.641251e+05,1.288997e+06,8.191263e+06,8.191263e+06,...,0.000000e+00,0.000000e+00,0,-4.060076,1.753123e+06,7.185247e+06,7.185247e+06,6,"[{'dt': 2018-03-26 20:00:00+00:00, 'amount': -...",0.0
2018-03-27 20:00:00+00:00,0.680969,-0.102664,0.346123,-0.049599,0.286452,2.244419,-1.433131e+05,1.145684e+06,7.827673e+06,7.827673e+06,...,0.000000e+00,0.000000e+00,0,-6.502873,1.288997e+06,8.191263e+06,8.191263e+06,7,"[{'dt': 2018-03-27 20:00:00+00:00, 'amount': 1...",0.0
2018-03-28 20:00:00+00:00,0.631398,-0.110219,0.236895,-0.052407,0.266222,2.238594,-7.020725e+05,4.436119e+05,8.454195e+06,8.454195e+06,...,0.000000e+00,0.000000e+00,0,-6.558945,1.145684e+06,7.827673e+06,7.827673e+06,8,"[{'dt': 2018-03-28 20:00:00+00:00, 'amount': 2...",0.0
2018-03-29 20:00:00+00:00,0.641898,-0.080222,0.388406,-0.040299,0.269279,2.269756,-5.534705e+05,-1.098586e+05,9.307634e+06,9.307634e+06,...,0.000000e+00,0.000000e+00,0,-4.295188,4.436119e+05,8.454195e+06,8.454195e+06,9,"[{'dt': 2018-03-29 20:00:00+00:00, 'amount': -...",0.0
2018-04-02 20:00:00+00:00,0.634514,-0.122994,0.403470,-0.061014,0.268083,2.264051,-6.945396e+05,-8.043981e+05,9.574459e+06,9.574459e+06,...,0.000000e+00,0.000000e+00,0,-5.873038,-1.098586e+05,9.307634e+06,9.307634e+06,10,"[{'dt': 2018-04-02 20:00:00+00:00, 'amount': -...",0.0


In [300]:
arima = pd.read_csv('arima_predictions.csv')
arima = arima.set_index('date')

In [301]:
mu = mean_historical_return(arima)
S = CovarianceShrinkage(arima).shrunk_covariance()
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()

In [302]:
weights

{'BIB': 0.040227135322093446,
 'BRZU': 0.115456726406399,
 'CURE': 0.020629216519574292,
 'DRN': 0.0,
 'EDC': 0.057326483801642465,
 'FAS': 0.01485412121296166,
 'FINU': 0.0408765920114711,
 'GOEX': 0.0,
 'INDL': 7.771323235899951e-10,
 'JNUG': 7.768568777285359e-10,
 'JPNL': 0.0,
 'KOLD': 0.03661120697732328,
 'LBJ': 0.11343587418424117,
 'MIDU': 0.0,
 'NUGT': 7.769559440389752e-10,
 'QLD': 0.05954479432446482,
 'RETL': 0.0,
 'ROM': 0.053461041645592525,
 'SCO': 7.768330444211447e-10,
 'SOXL': 0.057878501999427505,
 'SPXL': 0.00026114557008145617,
 'SVXY': 9.92480392495064e-10,
 'TECL': 0.0667326215076403,
 'TMF': 7.77066887885858e-10,
 'TNA': 0.0,
 'TQQQ': 0.07703778397988767,
 'UBR': 0.09977570709135583,
 'UDOW': 0.0,
 'UMDD': 0.0,
 'UPRO': 0.0,
 'URTY': 0.0,
 'USD': 0.058107746848446246,
 'UWM': 0.0,
 'XPP': 0.03698468183033609,
 'YINN': 0.050799006275232185}